<a href="https://colab.research.google.com/github/enoosoft/ai_fsec/blob/master/colab/Day_2_02_logistic_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 5: Logistic Classification

## Reminder: Logistic Regression

### Hypothesis

$$ H(X) = \frac{1}{1+e^{-W^T X}} $$

### Cost

$$ cost(W) = -\frac{1}{m} \sum y \log\left(H(x)\right) + (1-y) \left( \log(1-H(x) \right) $$

 - If $y \simeq H(x)$, cost is near 0.
 - If $y \neq H(x)$, cost is high.

### Weight Update via Gradient Descent

$$ W := W - \alpha \frac{\partial}{\partial W} cost(W) $$

 - $\alpha$: Learning rate

## Imports

In [144]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [145]:
# For reproducibility
torch.manual_seed(1)

## Training Data

In [146]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]] # 2x2 (국어공부시간, 수학공부시간)
y_data = [[0], [0], [0], [1], [1], [1]] # 합격=1/불합격=0

Consider the following classification problem: given the number of hours each student spent watching the lecture and working in the code lab, predict whether the student passed or failed a course. For example, the first (index 0) student watched the lecture for 1 hour and spent 2 hours in the lab session ([1, 2]), and ended up failing the course ([0]).

In [147]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

As always, we need these data to be in `torch.Tensor` format, so we convert them.

In [148]:
print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


## Computing the Hypothesis

$$ H(X) = \frac{1}{1+e^{-W^T X}} $$

PyTorch has a `torch.exp()` function that resembles the exponential function.

In [149]:
print('e^1 equals: ', torch.exp(torch.FloatTensor([1])))

e^1 equals:  tensor([2.7183])


We can use it to compute the hypothesis function conveniently.

In [150]:
W = torch.zeros((2, 1), requires_grad=True)  # 학습의 대상이다 = True
b = torch.zeros(1, requires_grad=True)       # 학습의 대상이다 = True

In [151]:
hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(W) + b)))

In [152]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
torch.Size([6, 1])


Or, we could use `torch.sigmoid()` function! This resembles the sigmoid function:

In [153]:
print('1/(1+e^{-1}) equals: ', torch.sigmoid(torch.FloatTensor([1])))

1/(1+e^{-1}) equals:  tensor([0.7311])


Now, the code for hypothesis function is cleaner.

In [154]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)

In [155]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward0>)
torch.Size([6, 1])


## Computing the Cost Function (Low-level)

$$ cost(W) = -\frac{1}{m} \sum y \log\left(H(x)\right) + (1-y) \left( \log(1-H(x) \right) $$

We want to measure the difference between `hypothesis` and `y_train`.

In [156]:
print(hypothesis)
print(y_train)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward0>)
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])


For one element, the loss can be computed as follows:

In [157]:
-(y_train[0] * torch.log(hypothesis[0]) + 
  (1 - y_train[0]) * torch.log(1 - hypothesis[0]))

tensor([0.6931], grad_fn=<NegBackward0>)

To compute the losses for the entire batch, we can simply input the entire vector.

In [158]:
losses = -(y_train * torch.log(hypothesis) + 
           (1 - y_train) * torch.log(1 - hypothesis))
print(losses)

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward0>)


Then, we just `.mean()` to take the mean of these individual losses.

In [159]:
cost = losses.mean()
print(cost)

tensor(0.6931, grad_fn=<MeanBackward0>)


## Computing the Cost Function with `F.binary_cross_entropy`

In reality, binary classification is used so often that PyTorch has a simple function called `F.binary_cross_entropy` implemented to lighten the burden.

In [160]:
F.binary_cross_entropy(hypothesis, y_train) # log cost 함수와 동일함

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward0>)

## Training with Low-level Binary Cross Entropy Loss

In [161]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [162]:
# 모델 초기화
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정 경사하강법
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 5000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b) # or .mm or @
    cost = -(y_train * torch.log(hypothesis) + 
             (1 - y_train) * torch.log(1 - hypothesis)).mean() # cost 함수

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 500 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/5000 Cost: 0.693147
Epoch  500/5000 Cost: 0.037261
Epoch 1000/5000 Cost: 0.019852
Epoch 1500/5000 Cost: 0.013562
Epoch 2000/5000 Cost: 0.010305
Epoch 2500/5000 Cost: 0.008311
Epoch 3000/5000 Cost: 0.006965
Epoch 3500/5000 Cost: 0.005994
Epoch 4000/5000 Cost: 0.005260
Epoch 4500/5000 Cost: 0.004687
Epoch 5000/5000 Cost: 0.004227


## Training with `F.binary_cross_entropy`

In [163]:
# 모델 초기화
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 10000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b) # or .mm or @
    cost = F.binary_cross_entropy(hypothesis, y_train) # cost 함수

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/10000 Cost: 0.693147
Epoch  100/10000 Cost: 0.134722
Epoch  200/10000 Cost: 0.080643
Epoch  300/10000 Cost: 0.057900
Epoch  400/10000 Cost: 0.045300
Epoch  500/10000 Cost: 0.037261
Epoch  600/10000 Cost: 0.031672
Epoch  700/10000 Cost: 0.027556
Epoch  800/10000 Cost: 0.024394
Epoch  900/10000 Cost: 0.021888
Epoch 1000/10000 Cost: 0.019852
Epoch 1100/10000 Cost: 0.018165
Epoch 1200/10000 Cost: 0.016743
Epoch 1300/10000 Cost: 0.015528
Epoch 1400/10000 Cost: 0.014479
Epoch 1500/10000 Cost: 0.013562
Epoch 1600/10000 Cost: 0.012755
Epoch 1700/10000 Cost: 0.012039
Epoch 1800/10000 Cost: 0.011400
Epoch 1900/10000 Cost: 0.010825
Epoch 2000/10000 Cost: 0.010305
Epoch 2100/10000 Cost: 0.009833
Epoch 2200/10000 Cost: 0.009403
Epoch 2300/10000 Cost: 0.009008
Epoch 2400/10000 Cost: 0.008646
Epoch 2500/10000 Cost: 0.008311
Epoch 2600/10000 Cost: 0.008002
Epoch 2700/10000 Cost: 0.007715
Epoch 2800/10000 Cost: 0.007447
Epoch 2900/10000 Cost: 0.007198
Epoch 3000/10000 Cost: 0.006965
Epoch 31

## Loading Real Data

In [164]:
import numpy as np

In [165]:
from google.colab import files
files.upload()

Saving data-03-diabetes.csv to data-03-diabetes.csv


{'data-03-diabetes.csv': b'-0.294118,0.487437,0.180328,-0.292929,0,0.00149028,-0.53117,-0.0333333,0\n-0.882353,-0.145729,0.0819672,-0.414141,0,-0.207153,-0.766866,-0.666667,1\n-0.0588235,0.839196,0.0491803,0,0,-0.305514,-0.492741,-0.633333,0\n-0.882353,-0.105528,0.0819672,-0.535354,-0.777778,-0.162444,-0.923997,0,1\n0,0.376884,-0.344262,-0.292929,-0.602837,0.28465,0.887276,-0.6,0\n-0.411765,0.165829,0.213115,0,0,-0.23696,-0.894962,-0.7,1\n-0.647059,-0.21608,-0.180328,-0.353535,-0.791962,-0.0760059,-0.854825,-0.833333,0\n0.176471,0.155779,0,0,0,0.052161,-0.952178,-0.733333,1\n-0.764706,0.979899,0.147541,-0.0909091,0.283688,-0.0909091,-0.931682,0.0666667,0\n-0.0588235,0.256281,0.57377,0,0,0,-0.868488,0.1,0\n-0.529412,0.105528,0.508197,0,0,0.120715,-0.903501,-0.7,1\n0.176471,0.688442,0.213115,0,0,0.132638,-0.608027,-0.566667,0\n0.176471,0.396985,0.311475,0,0,-0.19225,0.163962,0.2,1\n-0.882353,0.899497,-0.0163934,-0.535354,1,-0.102832,-0.726729,0.266667,0\n-0.176471,0.00502513,0,0,0,-0.105

In [166]:
xy = np.loadtxt('/content/data-03-diabetes.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1] # 행은 다가져오고 0~8까지 slicing
y_data = xy[:, [-1]] # 행은 다가져오고 9만 slicing
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [167]:
print(x_train[0:5])
print(y_train[0:5])

tensor([[-0.2941,  0.4874,  0.1803, -0.2929,  0.0000,  0.0015, -0.5312, -0.0333],
        [-0.8824, -0.1457,  0.0820, -0.4141,  0.0000, -0.2072, -0.7669, -0.6667],
        [-0.0588,  0.8392,  0.0492,  0.0000,  0.0000, -0.3055, -0.4927, -0.6333],
        [-0.8824, -0.1055,  0.0820, -0.5354, -0.7778, -0.1624, -0.9240,  0.0000],
        [ 0.0000,  0.3769, -0.3443, -0.2929, -0.6028,  0.2846,  0.8873, -0.6000]])
tensor([[0.],
        [1.],
        [0.],
        [1.],
        [0.]])


## Training with Real Data using low-level Binary Cross Entropy Loss

In [170]:
# 모델 초기화
W = torch.zeros((8, 1), requires_grad=True) # 8차원 입력
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 10000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b) # or .mm or @
    cost = -(y_train * torch.log(hypothesis) + (1 - y_train) * torch.log(1 - hypothesis)).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 10번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/10000 Cost: 0.693147
Epoch  100/10000 Cost: 0.480661
Epoch  200/10000 Cost: 0.473846
Epoch  300/10000 Cost: 0.472511
Epoch  400/10000 Cost: 0.472051
Epoch  500/10000 Cost: 0.471851
Epoch  600/10000 Cost: 0.471758
Epoch  700/10000 Cost: 0.471713
Epoch  800/10000 Cost: 0.471692
Epoch  900/10000 Cost: 0.471681
Epoch 1000/10000 Cost: 0.471676
Epoch 1100/10000 Cost: 0.471673
Epoch 1200/10000 Cost: 0.471672
Epoch 1300/10000 Cost: 0.471671
Epoch 1400/10000 Cost: 0.471671
Epoch 1500/10000 Cost: 0.471671
Epoch 1600/10000 Cost: 0.471671
Epoch 1700/10000 Cost: 0.471671
Epoch 1800/10000 Cost: 0.471671
Epoch 1900/10000 Cost: 0.471671
Epoch 2000/10000 Cost: 0.471671
Epoch 2100/10000 Cost: 0.471671
Epoch 2200/10000 Cost: 0.471671
Epoch 2300/10000 Cost: 0.471671
Epoch 2400/10000 Cost: 0.471671
Epoch 2500/10000 Cost: 0.471671
Epoch 2600/10000 Cost: 0.471671
Epoch 2700/10000 Cost: 0.471671
Epoch 2800/10000 Cost: 0.471671
Epoch 2900/10000 Cost: 0.471671
Epoch 3000/10000 Cost: 0.471671
Epoch 31

## Training with Real Data using `F.binary_cross_entropy`
- 실데이터에 비해 모델이 단순하기 때문에 학습결과가 좋지 않다(cost가 줄지 않는다)

In [171]:
# 모델 초기화
W = torch.zeros((8, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b) # or .mm or @
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 10번마다 로그 출력
    if epoch % 10 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 0.693147
Epoch   10/1000 Cost: 0.572727
Epoch   20/1000 Cost: 0.539493
Epoch   30/1000 Cost: 0.519708
Epoch   40/1000 Cost: 0.507066
Epoch   50/1000 Cost: 0.498539
Epoch   60/1000 Cost: 0.492549
Epoch   70/1000 Cost: 0.488209
Epoch   80/1000 Cost: 0.484985
Epoch   90/1000 Cost: 0.482543
Epoch  100/1000 Cost: 0.480661
Epoch  110/1000 Cost: 0.479189
Epoch  120/1000 Cost: 0.478023
Epoch  130/1000 Cost: 0.477088
Epoch  140/1000 Cost: 0.476331
Epoch  150/1000 Cost: 0.475711
Epoch  160/1000 Cost: 0.475198
Epoch  170/1000 Cost: 0.474771
Epoch  180/1000 Cost: 0.474411
Epoch  190/1000 Cost: 0.474107
Epoch  200/1000 Cost: 0.473846
Epoch  210/1000 Cost: 0.473622
Epoch  220/1000 Cost: 0.473428
Epoch  230/1000 Cost: 0.473259
Epoch  240/1000 Cost: 0.473111
Epoch  250/1000 Cost: 0.472980
Epoch  260/1000 Cost: 0.472864
Epoch  270/1000 Cost: 0.472761
Epoch  280/1000 Cost: 0.472669
Epoch  290/1000 Cost: 0.472586
Epoch  300/1000 Cost: 0.472511
Epoch  310/1000 Cost: 0.472444
Epoch  3

## Checking the Accuracy our our Model

After we finish training the model, we want to check how well our model fits the training set.

In [172]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)
print(hypothesis[:5])

tensor([[0.3555],
        [0.9564],
        [0.1934],
        [0.9623],
        [0.0706]], grad_fn=<SliceBackward0>)


We can change **hypothesis** (real number from 0 to 1) to **binary predictions** (either 0 or 1) by comparing them to 0.5.

In [173]:
prediction = hypothesis >= torch.FloatTensor([0.5])
print(prediction[:5])

tensor([[False],
        [ True],
        [False],
        [ True],
        [False]])


Then, we compare it with the correct labels `y_train`.

In [174]:
print(prediction[:5])
print(y_train[:5])

tensor([[False],
        [ True],
        [False],
        [ True],
        [False]])
tensor([[0.],
        [1.],
        [0.],
        [1.],
        [0.]])


In [175]:
correct_prediction = prediction.float() == y_train
print(correct_prediction[:5])

tensor([[True],
        [True],
        [True],
        [True],
        [True]])


Finally, we can calculate the accuracy by counting the number of correct predictions and dividng by total number of predictions.

In [180]:
accuracy = correct_prediction.sum().item() / len(correct_prediction)
print('The model has an accuracy of {:2.2f}% for the training set.'.format(accuracy * 100))

The model has an accuracy of 76.94% for the training set.


## Optional: High-level Implementation with `nn.Module`

In [181]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(8, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [182]:
model = BinaryClassifier()

In [183]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = model(x_train)

    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))


Epoch    0/1000 Cost: 0.691402 Accuracy 55.47%
Epoch   10/1000 Cost: 0.576110 Accuracy 66.93%
Epoch   20/1000 Cost: 0.541669 Accuracy 72.33%
Epoch   30/1000 Cost: 0.521164 Accuracy 76.28%
Epoch   40/1000 Cost: 0.508066 Accuracy 76.55%
Epoch   50/1000 Cost: 0.499238 Accuracy 77.08%
Epoch   60/1000 Cost: 0.493040 Accuracy 77.08%
Epoch   70/1000 Cost: 0.488553 Accuracy 77.08%
Epoch   80/1000 Cost: 0.485224 Accuracy 76.42%
Epoch   90/1000 Cost: 0.482704 Accuracy 76.42%
Epoch  100/1000 Cost: 0.480765 Accuracy 76.81%
Epoch  110/1000 Cost: 0.479250 Accuracy 76.94%
Epoch  120/1000 Cost: 0.478053 Accuracy 76.94%
Epoch  130/1000 Cost: 0.477094 Accuracy 76.94%
Epoch  140/1000 Cost: 0.476319 Accuracy 76.94%
Epoch  150/1000 Cost: 0.475686 Accuracy 76.94%
Epoch  160/1000 Cost: 0.475164 Accuracy 76.94%
Epoch  170/1000 Cost: 0.474730 Accuracy 77.08%
Epoch  180/1000 Cost: 0.474366 Accuracy 77.08%
Epoch  190/1000 Cost: 0.474058 Accuracy 77.08%
Epoch  200/1000 Cost: 0.473796 Accuracy 77.08%
Epoch  210/10